In [10]:
import sys
import csv
csv.field_size_limit(sys.maxsize)

from collections import defaultdict
import tqdm
import geopandas as gpd
from shapely.geometry import Point
import random
import matplotlib.pyplot as plt
from Bio import SeqIO
import datetime as dt
import os


In [3]:
def decimal_date(date):
    year = date.year
    if year == 2020:
        div = 366
    else:
        div = 365
    start = dt.date(year=year, month=1, day=1)
    decimal = (date - start).days/div

    dec_date = year + decimal
    
    return dec_date      
    

In [4]:
def recur_bit(min_x, min_y, max_x, max_y, polygon, pc, count):
    
    point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))

    if polygon.contains(point):
        return point
    else:
        count += 1
        return recur_bit(min_x, min_y, max_x, max_y, polygon, pc, count)
        
def find_point(polygon, pc):
    count = 0
    min_x, min_y, max_x, max_y = polygon.bounds
    point = recur_bit(min_x, min_y, max_x, max_y, polygon, pc, count)
    return point

In [6]:
## prep postcode data
pc_map = gpd.read_file("../../Data/UK_geog_data/England_postcode_districts.json")
pc_map.crs = "epsg:27700"
pc_map = pc_map.to_crs("epsg:4326")

dist_to_polygon = {}
for dist, geom in zip(pc_map["PostDist"], pc_map["geometry"]):
    dist_to_polygon[dist] = geom

dist_to_polygon["E20"] = dist_to_polygon["E15"] 

## five largest lineages

In [20]:
def make_input_files(intro_dir, wanted_intros,metadata, outdir):

    lin_to_taxa = defaultdict(list)
    all_taxa = set()
    tax_to_fullname = {}
    
    for intro_list in os.listdir(intro_dir):
        if intro_list.endswith(".tsv"):
            tree_number = intro_list.split(".")[0].split("_")[1][-1]
            relevant_intros = [i for i in wanted_intros if i.startswith(tree_number)]
            with open(os.path.join(intro_dir,intro_list)) as f:
                data = csv.DictReader(f, delimiter="\t")
                for line in data:
                    if f"{tree_number}_{line['lineage']}" in relevant_intros:
                        taxa = line['taxa'].split(";")
                        for i in taxa:
                            lin_to_taxa[f"{tree_number}_{line['lineage']}"].append(i.strip(" "))
                            all_taxa.add(i.strip(" ").split("|")[0])
                            tax_to_fullname[i.strip(" ").split("|")[0]] = i.strip(" ")
                    
    taxa_to_postcode = {}
    with open(metadata) as f:
        data = csv.DictReader(f)
        print(data.fieldnames)
        for line in tqdm.tqdm(data):
            if line['sequence_name'] in all_taxa:
                if line['outer_postcode'] != "":
                    taxa_to_postcode[line['sequence_name']] = line['outer_postcode']
                    
    tax_to_coord = {}
    for tax, pc in tqdm.tqdm(taxa_to_postcode.items()):
        try:
            polygon = dist_to_polygon[pc]
        except:
            continue
        point = find_point(polygon, pc)
        tax_to_coord[tax] = point
        
    ##make traits file
    for lin, taxa in lin_to_taxa.items():
        file_name = f'{outdir}/{lin}_traits.tsv'
        with open(file_name, 'w') as fw:
            fw.write("taxon\tlatitude\tlongitude\n")
            for tax in taxa:
                lookup = tax.split("|")[0]
                if lookup in tax_to_coord:
                    date_str = tax.split("|")[1]
                    date = decimal_date(dt.datetime.strptime(date_str, "%Y-%m-%d").date())

                    fw.write(f"{tax}\t{tax_to_coord[lookup].y}\t{tax_to_coord[lookup].x}\n")
                    
    ##write out jclusterfunk lists
    for lin, taxa in lin_to_taxa.items():
        with open(f"{outdir}/{lin}.csv", 'w') as fw:
            fw.write("name\n")
            for tax in taxa:
                lookup = tax.split("|")[0]
                if lookup in tax_to_coord:
                    fw.write(f'{tax}\n')

In [21]:
metadata = ##METADATA_FILE
intro_dir = ##TREE_FILES

wanted_intros = ["0_92", "0_190", "2_337", "0_67", "1_51", "0_322", "0_386"]  

make_input_files(intro_dir, wanted_intros,metadata, "../input_files/large_lineages/")

['sequence_name', 'secondary_identifier', 'sample_date', 'epi_week', 'country', 'adm1', 'adm2', 'outer_postcode', 'is_surveillance', 'is_community', 'is_hcw', 'is_travel_history', 'travel_history', 'lineage', 'lineages_version', 'lineage_conflict', 'lineage_ambiguity_score', 'scorpio_call', 'scorpio_support', 'scorpio_conflict']


582211it [00:02, 210196.69it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 25143/25143 [00:09<00:00, 2684.36it/s]


## small lineages

In [23]:
def make_input_files_small_lins(intro_list, wanted_intros,metadata, outdir):

    lin_to_taxa = defaultdict(list)
    all_taxa = set()
    tax_to_fullname = {}
    
    for intro_list in os.listdir(intro_dir):
        if intro_list.endswith(".tsv"):
            tree_number = intro_list.split(".")[0].split("_")[1][-1]
            relevant_intros = [i for i in wanted_intros if i.startswith(tree_number)]
            with open(os.path.join(intro_dir,intro_list)) as f:
                data = csv.DictReader(f, delimiter="\t")
                for line in data:
                    if f"{tree_number}_{line['lineage']}" in relevant_intros:
                        taxa = line['taxa'].split(";")
                        for i in taxa:
                            lin_to_taxa[f"{tree_number}_{line['lineage']}"].append(i.strip(" "))
                            all_taxa.add(i.strip(" ").split("|")[0])
                            tax_to_fullname[i.strip(" ").split("|")[0]] = i.strip(" ")
                    
    taxa_to_postcode = {}
    with open(metadata) as f:
        data = csv.DictReader(f)
        print(data.fieldnames)
        for line in tqdm.tqdm(data):
            if line['sequence_name'] in all_taxa:
                if line['outer_postcode'] != "":
                    taxa_to_postcode[line['sequence_name']] = line['outer_postcode']
                    
    tax_to_coord = {}
    for tax, pc in tqdm.tqdm(taxa_to_postcode.items()):
        try:
            polygon = dist_to_polygon[pc]
        except:
            continue
        point = find_point(polygon, pc)
        tax_to_coord[tax] = point
        
    ##make traits file
    file_name = f'{outdir}/traits.tsv'
    with open(file_name, 'w') as fw:
        fw.write("taxon\tlatitude\tlongitude\n")
        for lin, taxa in lin_to_taxa.items():
                for tax in taxa:
                    lookup = tax.split("|")[0]
                    if lookup in tax_to_coord:
                        fw.write(f"{tax}\t{tax_to_coord[lookup].y}\t{tax_to_coord[lookup].x}\n")


    ##write out jclusterfunk lists
    for lin, taxa in lin_to_taxa.items():
        with open(f"{outdir}/jclusterfunk_lists/{lin}.csv", 'w') as fw:
            fw.write("name\n")
            for tax in taxa:
                lookup = tax.split("|")[0]
                if lookup in tax_to_coord:
                    fw.write(f'{tax}\n')



In [ ]:
metadata = ##METADATA
intro_dir = ##TREE_FILES

wanted_intros = []
for intro_list in os.listdir(intro_dir):
    if intro_list.endswith(".tsv"):
        tree_number = intro_list.split(".")[0].split("_")[1][-1]
        with open(os.path.join(intro_dir,intro_list)) as f:
            data = csv.DictReader(f, delimiter="\t")
            for line in data:
                if f"{tree_number}_{line['lineage']}" not in wanted_intros:
                    taxa = line['taxa'].split(";")
                    if len(taxa) >= 5 and len(taxa) < 1500:
                        wanted_intros.append(f"{tree_number}_{line['lineage']}")

print(len(wanted_intros))       

make_input_files_small_lins(intro_dir, wanted_intros,metadata, "../input_files/small_lineages")

280
['sequence_name', 'secondary_identifier', 'sample_date', 'epi_week', 'country', 'adm1', 'adm2', 'outer_postcode', 'is_surveillance', 'is_community', 'is_hcw', 'is_travel_history', 'travel_history', 'lineage', 'lineages_version', 'lineage_conflict', 'lineage_ambiguity_score', 'scorpio_call', 'scorpio_support', 'scorpio_conflict']


582211it [00:02, 213127.39it/s]
 65%|████████████████████████████████████████████████████████████████████▏                                    | 15844/24417 [00:05<00:02, 3011.49it/s]